In [1]:
!python --version

Python 3.6.15


In [2]:
!python -m pip install torch==1.9.0

In [3]:
!pip install transformers

In [1]:
!pip install -U zero_shot_re

In [2]:
from transformers import AutoTokenizer
from zero_shot_re import RelTaggerModel, RelationExtractor

model = RelTaggerModel.from_pretrained("fractalego/fewrel-zero-shot")
tokenizer = AutoTokenizer.from_pretrained("fractalego/fewrel-zero-shot")

relations = ['noble title', 'founding date', 'occupation of a person']
extractor = RelationExtractor(model, tokenizer, relations)
ranked_rels = extractor.rank(text='John Smith received an OBE', head='John Smith', tail='OBE')
print(ranked_rels)

/home/gs/Info/IA/NLP/fewrel_zero_shot/venv/lib/python3.6/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of the model checkpoint at bert-large-uncased-whole-word-masking-finetuned-squad were not used when initializing BertModel: ['qa_outputs.weight', 'qa_outputs.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


[('noble title', 0.9003546684980392), ('occupation of a person', 0.0009881258010864258), ('founding date', 0.00013267993927001953)]


Le modèle n'est pas adapté pour distinguer le conteneur et le contenu:

In [3]:
relations = ['container', 'content', 'other relation']
extractor = RelationExtractor(model, tokenizer, relations)
cubes='the cubes'
box='the box'
ranked_rels = extractor.rank(text='the box must be filled with the cubes', head=box, tail=cubes)
print(ranked_rels)

[('container', 0.9636894315481186), ('content', 0.719708263874054), ('other relation', 0.3384249806404114)]


Le modèle est bon pour trouver les relations de contenance:

In [52]:
relations = ['container', 'other relation']
extractor = RelationExtractor(model, tokenizer, relations)

In [57]:
def contenance(text,item1,item2):
    #print(text,",",item1,",",item2)
    ranked_rels = extractor.rank(text=text, head=item1, tail=item2)
    print(ranked_rels)
    if ranked_rels[1][1]>0.5:
        return 'other relation'
    elif ranked_rels[0][1]>0.9:
        return 'container/content'
    else:
        return 'other relation'

In [58]:
text='You must put the cubes and the spheres in the boxes'
box='the boxes'
cube='the cubes'
sphere='the spheres'

In [59]:
print(contenance(text,box,cube))
print(contenance(text,cube,sphere))
print(contenance(text,box,sphere))

[('container', 0.9523085951805115), ('other relation', 0.2757440209388733)]
container/content
[('container', 0.8768596798181534), ('other relation', 0.3286207318305969)]
other relation
[('container', 0.9329218715429306), ('other relation', 0.16817760467529297)]
container/content


In [32]:
text='You must put the boxes and the spheres in the cubes'
box='the boxes'
cube='the cubes'
sphere='the spheres'

In [33]:
print(contenance(text,box,cube))
print(contenance(text,cube,sphere))
print(contenance(text,box,sphere))

container/content
container/content
other relation


In [34]:
text='the cubes must be filled with boxes and spheres'
box='boxes'
cube='the cubes'
sphere='spheres'

In [35]:
print(contenance(text,box,cube))
print(contenance(text,cube,sphere))
print(contenance(text,box,sphere))

container/content
container/content
other relation


In [36]:
# Bibliothèques nécessaires
from itertools import combinations
import spacy

nlp = spacy.load("en_core_web_lg")

In [37]:
def get_chunk(sent):
    doc=nlp(sent)
    nps=doc.noun_chunks
    sentences=[]
    for np1, np2 in combinations([np for np in nps], 2):
        sentences.append((sent,np1.text,np2.text))
    return sentences

get_chunk("In the beginning God created the heaven.")

[('In the beginning God created the heaven.', 'the beginning', 'God'),
 ('In the beginning God created the heaven.', 'the beginning', 'the heaven'),
 ('In the beginning God created the heaven.', 'God', 'the heaven')]

In [39]:
def get_rel(sent):
    relations={}
    for (sent,item1,item2) in get_chunk(sent):
        relations[(item1,item2)]=contenance(sent,item1,item2)
    return relations
        
get_rel('the boxes must be filled with cubes and spheres')

{('the boxes', 'cubes'): 'container/content',
 ('the boxes', 'spheres'): 'container/content',
 ('cubes', 'spheres'): 'other relation'}

In [53]:
get_rel("the treasur must")

[('container', 0.5033380687236786), ('other relation', 0.027729690074920654)]
[('container', 0.818322628736496), ('other relation', 0.29057759046554565)]
[('container', 0.8243522197008133), ('other relation', 0.23067349195480347)]
[('container', 0.36695683002471924), ('other relation', 0.019870400428771973)]
[('container', 0.3167724609375), ('other relation', 0.020469605922698975)]
[('container', 0.8032722026109695), ('other relation', 0.4039815664291382)]


{('the car', 'the road'): 'other relation',
 ('the car', 'the chest'): 'other relation',
 ('the car', 'a treasure'): 'other relation',
 ('the road', 'the chest'): 'other relation',
 ('the road', 'a treasure'): 'other relation',
 ('the chest', 'a treasure'): 'other relation'}